# Deploy on any 2 songs
- Use model to calculate embeddings (using eval mode specifically and with no gradient updating)
- NOTE: Right now, we have to have the input data in the correct format: a spectrogram/chromagram/tempogram (generically called "gram"). So for any deployment, we'll have to do preprocessing in the streamlit app for example. OR we can have a set of say 10-15 sample songs you can compare where we've already done all of the calculations.
- **Similarity values key:**
    - 0.5 to 1: Very similar. Perhaps the same song.
    - 0 to 0.5: Somewhat similar. Share some key characteristics
    - -1 to 0: Low to no similarity. Different songs.


In [ ]:
import torch

# Load the model
model = torch.load('full_model.pth')
model.eval()  # Set to evaluation mode


In [ ]:
# Move the model to the appropriate device (if using GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the model to evaluation mode
model.eval()

In [ ]:
# How to load the model later
#model = ResNetEmbedding()  # Make sure this matches the architecture you used

# Load the state dictionary into the model
#model.load_state_dict(torch.load('resnet18_model_weights.pth'))

# If using a GPU
#model.to(device)

In [ ]:
import librosa
import torch
import torch.nn.functional as F

def extract_embedding(model, audio_data_clip, sr=22050):
    # Load the audio file
    y, sr = librosa.load(audio_data_clip, sr=sr)
    
    # Convert raw audio to mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    # Convert to tensor and move to the appropriate device
    mel_tensor = torch.tensor(mel_spectrogram_db, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
    
    # Get the embedding from the model
    with torch.no_grad():
        embedding = model(mel_tensor)
    
    # Normalize the embedding
    embedding = F.normalize(embedding, p=2, dim=1)
    
    return embedding


In [ ]:
import torch.nn.functional as F

def compute_cosine_similarity(embedding1, embedding2):
    # Compute cosine similarity
    cosine_sim = F.cosine_similarity(embedding1, embedding2)
    return cosine_sim.item()  # Convert to a Python float


In [ ]:
# Paths to the audio files (these should be the paths to your actual song files)
anchor = '/Users/reggiebain/erdos/song-similarity-erdos-old/data/coversongs/covers32k/A_Whiter_Shade_Of_Pale/annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale.mp3'
positive = '/Users/reggiebain/erdos/song-similarity-erdos-old/data/coversongs/covers32k/A_Whiter_Shade_Of_Pale/procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of_Pale.mp3'
negative = '/Users/reggiebain/erdos/song-similarity-erdos-old/data/coversongs/covers32k/Abracadabra/steve_miller_band+Steve_Miller_Band_Live_+09-Abracadabra.mp3'

# Extract embeddings for both songs
embedding1 = extract_embedding(model, audio_clip1)
embedding2 = extract_embedding(model, audio_clip2)

# Calculate cosine similarity between the two embeddings
similarity = compute_cosine_similarity(embedding1, embedding2)

print(f"Cosine Similarity between the two songs: {similarity:.4f}")


### Search Spotify for Track to Analyze

In [ ]:
# Load stuff from .env file
env_vars = dotenv_values('.env')

client_credentials_manager = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET"),
)
# Get Spotify api client and apply to df
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Function to get the preview URL of a song based on artist name and song title
#@retry(wait=wait_exponential(multiplier=1, min=4, max=60), stop=stop_after_attempt(10))
def search_track(artist_name, track_name, sp, rate_limit = 1.0):
    # Search for the track
    result = sp.search(q=f'artist:{artist_name} track:{track_name}', type='track', limit=1)
    if result['tracks']['items']:
        # Return the preview URL if found
        return result['tracks']['items'][0]['preview_url']
    print('Arist/Track not found...')
    return None    

In [ ]:
track1_name = 'Rosanna'
track1_artist = 'Toto'
track2_name = 'Rosanna'
track2_artist = 'Toto'
track1 = search_track(track1_artist, track1_name, sp)
track2 = search_track(track1_artist, track1_name, sp)

In [ ]:
artist_name = 'Toto'
track_name = 'Rosanna'
result = sp.search(q=f'artist:{artist_name} track:{track_name}', type='track', limit=1)
result['tracks']['items'][0]['preview_url']